Load lib

In [2]:
import matplotlib.pyplot as plt
import gym
from gym.wrappers.monitoring.video_recorder import VideoRecorder
from os import path
from moviepy.editor import VideoClip
from moviepy.video.io.bindings import mplfig_to_npimage
import copy
import os
import numpy as np
import random
import pickle #rick
%matplotlib inline
fps=50 #remove q_table file if you reduce this value
path_project = os.path.abspath(os.path.join("__file__", ".."))
q_object=path_project+"/q_table"
video_after = os.path.join(path_project, "after_training.mp4")
video_before= os.path.join(path_project, "before_training.mp4")
max_steps = 100*fps #remove q_table file if you reduce this value

In [ ]:
Create Q table

In [3]:

env = gym.make("LunarLander-v2",render_mode="rgb_array")

alpha = 0.7 #learning rate
discount_factor = 0.8                         
max_epsilon = 0.5
min_epsilon = 0.001         
decay = 0.01        
train_episodes = 5000

print("we will run "+str(train_episodes)+" game with a maximum of "+str(max_steps)+" action")



#STEP 1 - load Q-table or create it

if path.isfile(q_object):
    print('load qtable')
    fileQ=open(q_object, 'rb')
    Q = pickle.load(fileQ)
    fileQ.close()
    while Q.size/env.action_space.n<max_steps:
        Q=np.vstack([Q,np.zeros(env.action_space.n)])
else:
    Q = np.zeros((max_steps, env.action_space.n))

q_object=path_project+"/q_table"
fileQ=open(q_object, 'wb')
#Training the agent

#Creating lists to keep track of reward and epsilon values
total_training_rewards=0
training_rewards = []  
epsilons = []
maxi=0

for episode in range(train_episodes):
    #Reseting the environment each time as per requirement
    state = env.reset(seed=42)  
    lst=[]

    epsilon= min_epsilon+(max_epsilon-min_epsilon)*np.exp(-decay*episode)   
    for step in range(max_steps-1):
        #tmpepsi=(max_epsilon-(max_epsilon-min_epsilon)*np.exp(-decay*step) +min_epsilon+(max_epsilon-min_epsilon)*np.exp(-decay*episode))/2
        ### STEP 2: SECOND option for choosing the initial action - exploit     
        #If the random number is larger than epsilon: employing exploitation 
        #and selecting best action 
        if random.uniform(0, 1) > epsilon:
  
            action = np.argmax(Q[step,:])      
            
        ### STEP 2: FIRST option for choosing the initial action - explore       
        #Otherwise, employing exploration: choosing a random action 
        else:
            action = env.action_space.sample()
    
            
        ### STEPs 3 & 4: performing the action and getting the reward     
        #Taking the action and getting the reward and outcome state
        new_state, reward, done,trunc, info = env.step(action)
        

        ### STEP 5: update the Q-table
        #Updating the Q-table using the Bellman equation
        Q[step, action] = Q[step, action]*(1-alpha) + alpha * (reward + discount_factor * np.max(Q[step+1, :])) 
        state=new_state
        total_training_rewards+=reward
        if done: break

    if episode%100==0:
        print("Epoch:"+str(episode+1)+'/'+str(train_episodes)+" epsi:"+str(epsilon))
    #Cutting down on exploration by reducing the epsilon 
    #epsilon= min_epsilon+(max_epsilon-min_epsilon)*np.exp(-decay*episode) 

     #Adding the total reward and reduced epsilon values
    training_rewards.append(total_training_rewards)
    
    if total_training_rewards > maxi:
        maxi=total_training_rewards
        print("Current best score: "+str(total_training_rewards))
   
        
    total_training_rewards=0
    epsilons.append(epsilon)
#save q table
pickle.dump(Q,fileQ)
fileQ.close() 
print("Max score:"+str(maxi))
print("Standard deviation: "+str(np.std(training_rewards)))
print ("Mean score: " + str(sum(training_rewards)/train_episodes))

we will run 5000 game with a maximum of 5000 action
Epoch:1/5000 epsi:0.5
Current best score: 11.791963532558427
Current best score: 39.79050973756614
Epoch:101/5000 epsi:0.4525138715999438
Epoch:201/5000 epsi:0.40954664578591293
Current best score: 56.68955265625712
Epoch:301/5000 epsi:0.3706682921201772
Epoch:401/5000 epsi:0.33548970297178404
Epoch:501/5000 epsi:0.3036587991966041
Epoch:601/5000 epsi:0.2748570064109192
Epoch:701/5000 epsi:0.24879606659191333
Epoch:801/5000 epsi:0.22521515309449355
Epoch:901/5000 epsi:0.20387826021055896
Epoch:1001/5000 epsi:0.18457184114454972
Epoch:1101/5000 epsi:0.1671026707653417
Epoch:1201/5000 epsi:0.15129591174418885
Epoch:1301/5000 epsi:0.13699336472397228
Epoch:1401/5000 epsi:0.12405188500686161
Epoch:1501/5000 epsi:0.11234194991406649
Epoch:1601/5000 epsi:0.10174636247933304
Epoch:1701/5000 epsi:0.0921590785023146
Epoch:1801/5000 epsi:0.08348414522257168
Epoch:1901/5000 epsi:0.07563474099209488
Epoch:2001/5000 epsi:0.06853230633506974
Epoch:

In [3]:


env = gym.make("LunarLander-v2",render_mode="rgb_array")

"""Lets see some good gameplay"""
def make_frame(t):
    """Returns an image of the frame for time t."""
    # ... create the frame with any library here ...
    
    return lst[int(t*fps)] # (Height x Width x 3) Numpy array
state=env.reset(seed=42)
lst=[]
done = False
if path.isfile(q_object):
    fileQ=open(q_object, 'rb')
    Q = pickle.load(fileQ)
    fileQ.close()
i=0
check=0
while i<max_steps-1 :
    if done: break
    lst.append(copy.deepcopy(env.render()))
    state, reward, done, _,_ =env.step(np.argmax(Q[i,:])) # take the best action
    check+=reward
    i+=1
    
print("Score_IA:"+str(check))
animation = VideoClip(make_frame, duration=len(lst)//fps) # T-second clip
# export as a video file
animation.write_videofile(video_after, fps=fps)

env.close()
        

Score_IA:-49.64727006865408
Moviepy - Building video /Users/florian/Desktop/atari/after_training.mp4.
Moviepy - Writing video /Users/florian/Desktop/atari/after_training.mp4



Moviepy - Done !
Moviepy - video ready /Users/florian/Desktop/atari/after_training.mp4


Mesure le score avec une strategie prise au hasard:

In [6]:

env = gym.make("LunarLander-v2",render_mode="rgb_array")

alpha = 0.7 #learning rate
      
decay = 0.01        
train_episodes = 1000

print("we will run "+str(train_episodes)+" game with a maximum of "+str(max_steps)+" action")




#Creating lists to keep track of reward and epsilon values
total_training_rewards=0
training_rewards = []  
epsilons = []


for episode in range(train_episodes):
    #Reseting the environment each time as per requirement
    state = env.reset(seed=42)  
    lst=[]
    for step in range(max_steps-1):

        action = env.action_space.sample()
    
        new_state, reward, done,trunc, info = env.step(action)
        state=new_state
        
        total_training_rewards+=reward
        if done: break

    if episode%100==0:
        print("Epoch:"+str(episode+1)+'/'+str(train_episodes))
     #Adding the total reward and reduced epsilon values
    training_rewards.append(total_training_rewards)
    total_training_rewards=0
    


print("Standard deviation: "+str(np.std(training_rewards)))
print ("Mean score: " + str(sum(training_rewards)/train_episodes))

we will run 1000 game with a maximum of 5000 action
Epoch:1/1000
Epoch:101/1000
Epoch:201/1000
Epoch:301/1000
Epoch:401/1000
Epoch:501/1000
Epoch:601/1000
Epoch:701/1000
Epoch:801/1000
Epoch:901/1000
Standard deviation: 115.15339223247264
Mean score: -212.27073895705442
